From publications DOI: <a href="http://doi.org/10.1364/OPTICA.487795">http://doi.org/10.1364/OPTICA.487795</a> and DOI: <a href="https://doi.org/10.1038/s41598-023-29762-y">https://doi.org/10.1038/s41598-023-29762-y</a>

In [ ]:
# Copyright (c) 2024, ETH Zurich

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
rave_sim_dir = Path('<PATH/TO/rave-sim>')
simulations_dir = Path('<PATH/WHERE/TO/STORE/SIMULATION/>')
scratch_dir = simulations_dir

In [ ]:
sys.path.insert(0, str(rave_sim_dir / "big-wave"))
import multisim
import config
import util

In [ ]:
config_dict = {
    "sim_params": {
        "N": 2**22,
        "dx": 3.1e-10,
        "z_detector": 1.77,
        "detector_size": 2e-4,
        "detector_pixel_size_x": 1e-6,
        "detector_pixel_size_y": 1,
        "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
    },
    "use_disk_vector": False,
    "save_final_u_vectors": False,
    "multisource": {
        "type": "points",
        "energy_range": [46000.0, 47000.0],
        "x_range": [-1e-6, 1e-6],
        "z": 0.0,
        "nr_source_points": 3,
        "seed": 1,
    },
    "elements": [
        {
            "type": "grating",
            "pitch": 4.2e-6,
            "dc": [0.5, 0.5],
            "z_start": 0.1,
            "thickness": 140e-6,
            "nr_steps": 8,
            "x_positions": [0.0],
            "substrate_thickness": (370 - 140) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": ["Au", 19.32],
            "mat_substrate": ["Si", 2.34],
        },
        {
            "type": "grating",
            "pitch": 4.2e-6,
            "dc": [0.5, 0.5],
            "z_start": 0.918,
            "thickness": 59e-6,
            "nr_steps": 8,
            "x_positions": [0.0],
            "substrate_thickness": (200 - 59) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": None,
            "mat_substrate": ["Si", 2.34],
        },
        {
            "type": "grating",
            "pitch": 4.2e-6,
            "dc": [0.5, 0.5],
            "z_start": 1.736,
            "thickness": 154e-6,
            "nr_steps": 8,
            "x_positions": np.linspace(0, 4.2e-6, 5, endpoint=False).tolist(),
            "substrate_thickness": (370 - 154) * 1e-6,
            "mat_a": ["Si", 2.34],
            "mat_b": ["Au", 19.32],
            "mat_substrate": ["Si", 2.34],
        },
    ],
}

print("N: ", config_dict["sim_params"]["N"])

In [ ]:
sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)

In [ ]:
computed = config.load(Path(sim_path / 'computed.yaml'))

print("cutoff angles:", computed['cutoff_angles'])
print("source points:", computed['source_points'])

In [ ]:
# Run this in a for loop to simulate all source points or
# alternatively run the source points as individual euler
# jobs
multisim.run_single_simulation(sim_path, 0, scratch_dir, save_keypoints_path=scratch_dir)

In [ ]:
wavefronts = util.load_wavefronts_filtered(sim_path, x_range=(-4e-7, 0))
print("nr sources loaded:", len(wavefronts))

wf = wavefronts[0][0]
print("nr phase steps:", wf.shape[0])
print("nr detector pixels:", wf.shape[1])

In [ ]:
sp = config_dict["sim_params"]
detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
plt.plot(detector_x, wf[0])

In [ ]:
full_x = util.full_x_vector(sp["N"], sp["dx"])
kp = util.load_keypoints(scratch_dir)

In [ ]:
plt.plot(full_x, np.abs(kp[4]) ** 2)
plt.xlim(-3e-5, 3e-5)

for i in range(-50, 50):
    plt.axvline(i * 4.2e-6)

plt.show()

## History

To see the interference pattern in empty space, we can record slices throughout the simulation and then plot them. `run_single_simulation` takes an optional argument `history_dz` defining the resolution with which we record the history.

Note that the history is not necessarily recorded with a constant z-spacing. Inside gratings and samples, one slice is recorded for every step. The history also records a list of z-coordinates at which the slices were recorded, which we can use for plotting.

In [ ]:
multisim.run_single_simulation(sim_path, 1, scratch_dir, save_keypoints_path=None, history_dz=0.02)

In [ ]:
# Path to the directory for the source with index 1
source_dir = multisim.get_sub_dir(sim_path, 1)

hist_x = np.load(source_dir / "history_x.npy")
hist_z = np.load(source_dir / "history_z.npy")
hist = np.load(source_dir / "history.npy")
plt.pcolormesh(
    hist_z,
    hist_x,
    hist,
    cmap="Greys_r",
    vmin=0,
    vmax=1e-6,
    shading="nearest",
)
plt.xlabel("z (m)")
plt.ylabel("x (m)")